In [1]:
#Comprobamos la existencia de los dataset
!ls /dataset

2.1  googleplaystoreV3.csv  texto.txt


In [2]:
#Creamos un directorio para almacenar los datos de entrada
!hadoop fs -mkdir /user/root/input

mkdir: `/user/root/input': File exists


In [3]:
#Copiamos el fichero texto.txt a HDFS
!hadoop fs -copyFromLocal /dataset/texto.txt /user/root/input

copyFromLocal: `/user/root/input/texto.txt': File exists


In [4]:
#Comprobamos que se ha importado correctamente y vemos la información 
!hadoop fs -ls /user/root/input

Found 3 items
drwxr-xr-x   - root supergroup          0 2022-11-03 19:54 /user/root/input/2.1
-rw-r--r--   3 root supergroup     664067 2022-11-03 19:54 /user/root/input/googleplaystoreV3.csv
-rw-r--r--   2 root supergroup         92 2022-11-03 19:54 /user/root/input/texto.txt


In [5]:
#Mostramos el contenido de texto.txt
!hadoop fs -cat /user/root/input/texto.txt

Esto es un ejemplo de programación mapreduce.
Esto es un ejemplo de contador de palabras. 


La programación MapReduce para Hadoop puede hacerse utilizando una gran variedad de lenguajes de programación. Por su utilidad en el análisis de datos y su sencillez y rápido aprendizaje utilizaremos Python. 

In [6]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys

#Leemos cada linea de la entrada estandar
for line in sys.stdin:  
    #Dividimos en palabras cada linea
    words = line.split()  
    #Iteramos sobre cada palabra contenida en words
    for word in words:    
        #Escribimos por salida estandar el par <Clave,Valor>
        #En este caso la Clave será la palabra en cuestión y el valor será 1
        print(f"{word}\t1")

Overwriting mapper.py


In [7]:
#Para que el script de Python se ejecute, necesita permisos de ejecución. Probamos a ejecutar en local.
!chmod +x mapper.py
!cat /dataset/texto.txt | ./mapper.py

Esto	1
es	1
un	1
ejemplo	1
de	1
programación	1
mapreduce.	1
Esto	1
es	1
un	1
ejemplo	1
de	1
contador	1
de	1
palabras.	1


In [8]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys

curr_word = None
curr_count = 0

# Procesamos cada Clave-Valor que produce la funcion map
for line in sys.stdin:

    #Dividimos la clave y el valor
    word, count = line.split('\t')

    #Convertimos a entero el valor
    count = int(count)

    #Si la palabra actual es la misma que la previa, incrementamos el contador
    #en otro caso imprimimos la palabra en cuestion con el contador
    if word == curr_word:
        curr_count += count
    else:
        if curr_word:   
            print (f"{curr_word}\t{curr_count}")
        
        #Actualizamos las variables a la nueva palabra
        curr_word = word
        curr_count = count

#Imprimimos la ultima palabra
if curr_word == word:
      print (f"{curr_word}\t{curr_count}")

Overwriting reducer.py


In [9]:
#Otorgamos permisos de ejecución al script reducer y probamos el programa completo en local. 
#El comando sort de unix hace de fase sort&shuffle
!chmod +x reducer.py
!cat /dataset/texto.txt | ./mapper.py | sort | ./reducer.py

Esto	2
contador	1
de	3
ejemplo	2
es	2
mapreduce.	1
palabras.	1
programación	1
un	2


In [10]:
#Ejecutamos el programa MapReduce en el clúster Hadoop. 
!mapred streaming \
  -input /user/root/input/texto.txt \
  -output /user/root/output \
  -file /media/notebooks/Leccion1.1/reducer.py \
  -file /media/notebooks/Leccion1.1/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py

2022-11-03 19:56:07,964 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
File: file:/media/notebooks/Leccion1.1/reducer.py does not exist.
Try -help for more information
Streaming Command Failed!


In [11]:
#El resultado se almacena en el directorio especificado en el comando.
#Se crea un archivo _SUCCESS que indica la correcta ejecución del programa.
#El resultado se almacena en el fichero part-00000
!hadoop fs -ls /user/root/output

ls: `/user/root/output': No such file or directory


In [12]:
#Comprobamos el resultado 
!hadoop fs -cat /user/root/output/*

cat: `/user/root/output/*': No such file or directory
